# Scraper for ICASSP from IEEE

> "From a pre-extracted page, content seems to be fetched separately"

- hidden: true
- toc: false
- badges: true
- categories: [scraper, icassp]

In [2]:
from bs4 import BeautifulSoup

The temporary file below is extracted from [IEEE](https://ieeexplore.ieee.org/xpl/conhome/1000002/all-proceedings)

In [20]:
with open("/tmp/icassp") as inf:
    html = inf.read()

In [21]:
soup = BeautifulSoup(html, 'html.parser')

In [22]:
top = soup.find("div", {"class": "issue-list-container"})

In [23]:
import re

In [59]:
data = []
for li in top.find_all("li", {"_ngcontent-ftl-c305": ""}):
    raw = li.text.strip()
    a_tag = li.find("a")
    href = a_tag.attrs["href"]
    title = a_tag.text.strip()
    m1 = re.search("^ICASSP (20\d\d).*", raw)
    m2 = re.search(".*((?:20|19)\d\d) (?:IEEE )?International.*", raw)
    m3 = re.search(".*ICASSP '(\d\d).*", raw)
    if m1:
        year = m1.group(1)
    elif m2:
        year = m2.group(1)
    elif m3:
        year = m3.group(1)
        # Why yes, in this, the Year of Our Lord, 2022,
        # writing this scraper has meant having to do
        # Y2K compensation
        if year.startswith("0"):
            year = "20" + year
        else:
            year = "19" + year
    elif raw == "International Conference on Acoustics, Speech, and Signal Processing":
        year = "1990"
    elif raw == "International Conference on Acoustics, Speech, and Signal Processing,":
        year == "1989"
    elif raw.startswith("ICASSP-88.,"):
        year = "1988"
    loc_tag = li.find("span", {"_ngcontent-ftl-c305": ""})
    if loc_tag:
        loc = loc_tag.text.strip().replace("Location: ", "")
    data.append([href, title, year, loc])

In [60]:
count = 1
data.reverse()
for item in data:
    item.append(str(count))
    count += 1
data.reverse()

In [62]:
template = "https://ieeexplore.ieee.org/rest/publication/home/metadata?pubid="

In [63]:
import requests

In [64]:
raw_json = requests.get(template + "9413349")


In [66]:
raw_json.text

'<HTML><HEAD><TITLE>Error</TITLE></HEAD><BODY>\nAn error occurred while processing your request.<p>\nReference&#32;&#35;30&#46;64aa2617&#46;1670514466&#46;8894ff4\n</BODY></HTML>\n'

Bah. Ok, API is a waste of time: limit is too low, registering for a key is a bit annoying for a one-time operation.

In [67]:
DOIs = """
10.1109/ICASSP43922.2022
10.1109/ICASSP39728.2021
DOI: 10.1109/ICASSP40776.2020
DOI: 10.1109/ICASSP35589.2019
DOI: 10.1109/ICASSP34228.2018
DOI: 10.1109/ICASSP31846.2017

DOI: 10.1109/ICASSP17257.2015
DOI: 10.1109/ICASSP18874.2014
DOI: 10.1109/ICASSP16080.2013
DOI: 10.1109/ICASSP15465.2012
DOI: 10.1109/ICASSP15948.2011
DOI: 10.1109/ICASSP15600.2010
DOI: 10.1109/ICASSP13629.2009
DOI: 10.1109/ICASSP12235.2008
DOI: 10.1109/ICASSP10710.2007
DOI: 10.1109/ICASSP10488.2006
DOI: 10.1109/ICASSP8829.2005
DOI: 10.1109/ICASSP.2004
DOI: 10.1109/ICASSP.2003
DOI: 10.1109/ICASSP.2002
DOI: 10.1109/ICASSP.2001
DOI: 10.1109/ICASSP.2000
DOI: 10.1109/ICASSP.1999
DOI: 10.1109/ICASSP.1998
DOI: 10.1109/ICASSP.1997
DOI: 10.1109/ICASSP.1996
DOI: 10.1109/ICASSP.1995
DOI: 10.1109/ICASSP.1994
DOI: 10.1109/ICASSP.1993
DOI: 10.1109/ICASSP.1992
DOI: 10.1109/ICASSP.1991
DOI: 10.1109/ICASSP.1990
DOI: 10.1109/ICASSP.1989
DOI: 10.1109/ICASSP.1988
DOI: 10.1109/ICASSP.1987
DOI: 10.1109/ICASSP.1986
DOI: 10.1109/ICASSP.1985
DOI: 10.1109/ICASSP.1984
DOI: 10.1109/ICASSP.1983
DOI: 10.1109/ICASSP.1982
DOI: 10.1109/ICASSP.1981
DOI: 10.1109/ICASSP.1980
DOI: 10.1109/ICASSP.1979
DOI: 10.1109/ICASSP.1978
DOI: 10.1109/ICASSP.1977
DOI: 10.1109/ICASSP.1976
"""

In [68]:
doi_dict = {}
for line in DOIs.replace("DOI: ", "").split("\n"):
    if line == "":
        continue
    parts = line.split(".")
    doi_dict[parts[-1]] = line

In [71]:
output = []
output.append("| Year | Ordinal | DOI | Issue | Location |")
output.append("|------|---------|-----|-------|----------|")
for item in data:
    #[href, title, year, loc, ord]
    if item[2] == "2016":
        doi = ""
    else:
        doi = doi_dict[item[2]]
    output.append(f"| {item[2]} | {item[4]} | {doi} | [{item[1]}](https://ieeexplore.ieee.org/{item[0]}) | {item[3]} |")


In [73]:
from IPython.display import display, Markdown

In [74]:
display(Markdown("\n".join(output)))

| Year | Ordinal | DOI | Issue | Location |
|------|---------|-----|-------|----------|
| 2022 | 47 | 10.1109/ICASSP43922.2022 | [ICASSP 2022 - 2022 IEEE International Conference on Acoustics, Speech and Signal Processing (ICASSP)](https://ieeexplore.ieee.org//xpl/conhome/9745891/proceeding) | Singapore, Singapore |
| 2021 | 46 | 10.1109/ICASSP39728.2021 | [ICASSP 2021 - 2021 IEEE International Conference on Acoustics, Speech and Signal Processing (ICASSP)](https://ieeexplore.ieee.org//xpl/conhome/9413349/proceeding) | Toronto, ON, Canada |
| 2020 | 45 | 10.1109/ICASSP40776.2020 | [ICASSP 2020 - 2020 IEEE International Conference on Acoustics, Speech and Signal Processing (ICASSP)](https://ieeexplore.ieee.org//xpl/conhome/9040208/proceeding) | Barcelona, Spain |
| 2019 | 44 | 10.1109/ICASSP35589.2019 | [ICASSP 2019 - 2019 IEEE International Conference on Acoustics, Speech and Signal Processing (ICASSP)](https://ieeexplore.ieee.org//xpl/conhome/8671773/proceeding) | Brighton, UK |
| 2018 | 43 | 10.1109/ICASSP34228.2018 | [2018 IEEE International Conference on Acoustics, Speech and Signal Processing (ICASSP)](https://ieeexplore.ieee.org//xpl/conhome/8450881/proceeding) | Calgary, AB, Canada |
| 2017 | 42 | 10.1109/ICASSP31846.2017 | [2017 IEEE International Conference on Acoustics, Speech and Signal Processing (ICASSP)](https://ieeexplore.ieee.org//xpl/conhome/7943262/proceeding) | New Orleans, LA, USA |
| 2016 | 41 |  | [2016 IEEE International Conference on Acoustics, Speech and Signal Processing (ICASSP)](https://ieeexplore.ieee.org//xpl/conhome/7465907/proceeding) | Shanghai, China |
| 2015 | 40 | 10.1109/ICASSP17257.2015 | [2015 IEEE International Conference on Acoustics, Speech and Signal Processing (ICASSP)](https://ieeexplore.ieee.org//xpl/conhome/7158221/proceeding) | South Brisbane, QLD, Australia |
| 2014 | 39 | 10.1109/ICASSP18874.2014 | [2014 IEEE International Conference on Acoustics, Speech and Signal Processing (ICASSP)](https://ieeexplore.ieee.org//xpl/conhome/6844297/proceeding) | Florence, Italy |
| 2013 | 38 | 10.1109/ICASSP16080.2013 | [2013 IEEE International Conference on Acoustics, Speech and Signal Processing](https://ieeexplore.ieee.org//xpl/conhome/6619549/proceeding) | Vancouver, BC, Canada |
| 2012 | 37 | 10.1109/ICASSP15465.2012 | [2012 IEEE International Conference on Acoustics, Speech and Signal Processing (ICASSP)](https://ieeexplore.ieee.org//xpl/conhome/6268628/proceeding) | Kyoto, Japan |
| 2011 | 36 | 10.1109/ICASSP15948.2011 | [2011 IEEE International Conference on Acoustics, Speech and Signal Processing (ICASSP)](https://ieeexplore.ieee.org//xpl/conhome/5916934/proceeding) | Prague, Czech Republic |
| 2010 | 35 | 10.1109/ICASSP15600.2010 | [2010 IEEE International Conference on Acoustics, Speech and Signal Processing](https://ieeexplore.ieee.org//xpl/conhome/5487364/proceeding) | Dallas, TX, USA |
| 2009 | 34 | 10.1109/ICASSP13629.2009 | [2009 IEEE International Conference on Acoustics, Speech and Signal Processing](https://ieeexplore.ieee.org//xpl/conhome/4912736/proceeding) | Taipei, Taiwan |
| 2008 | 33 | 10.1109/ICASSP12235.2008 | [2008 IEEE International Conference on Acoustics, Speech and Signal Processing](https://ieeexplore.ieee.org//xpl/conhome/4505270/proceeding) | Las Vegas, NV, USA |
| 2007 | 32 | 10.1109/ICASSP10710.2007 | [2007 IEEE International Conference on Acoustics, Speech and Signal Processing - ICASSP '07](https://ieeexplore.ieee.org//xpl/conhome/4216989/proceeding) | Honolulu, HI, USA |
| 2006 | 31 | 10.1109/ICASSP10488.2006 | [2006 IEEE International Conference on Acoustics Speech and Signal Processing Proceedings](https://ieeexplore.ieee.org//xpl/conhome/11024/proceeding) | Toulouse, France |
| 2005 | 30 | 10.1109/ICASSP8829.2005 | [Proceedings. (ICASSP '05). IEEE International Conference on Acoustics, Speech, and Signal Processing, 2005.](https://ieeexplore.ieee.org//xpl/conhome/9711/proceeding) | Philadelphia, PA, USA |
| 2004 | 29 | 10.1109/ICASSP.2004 | [2004 IEEE International Conference on Acoustics, Speech, and Signal Processing](https://ieeexplore.ieee.org//xpl/conhome/9248/proceeding) | Montreal, QC, Canada |
| 2003 | 28 | 10.1109/ICASSP.2003 | [2003 IEEE International Conference on Acoustics, Speech, and Signal Processing, 2003. Proceedings. (ICASSP '03).](https://ieeexplore.ieee.org//xpl/conhome/8535/proceeding) | Hong Kong, China |
| 2002 | 27 | 10.1109/ICASSP.2002 | [2002 IEEE International Conference on Acoustics, Speech, and Signal Processing](https://ieeexplore.ieee.org//xpl/conhome/7874/proceeding) | Orlando, FL, USA |
| 2001 | 26 | 10.1109/ICASSP.2001 | [2001 IEEE International Conference on Acoustics, Speech, and Signal Processing. Proceedings (Cat. No.01CH37221)](https://ieeexplore.ieee.org//xpl/conhome/7486/proceeding) | Salt Lake City, UT, USA |
| 2000 | 25 | 10.1109/ICASSP.2000 | [2000 IEEE International Conference on Acoustics, Speech, and Signal Processing. Proceedings (Cat. No.00CH37100)](https://ieeexplore.ieee.org//xpl/conhome/6939/proceeding) | Istanbul, Turkey |
| 1999 | 24 | 10.1109/ICASSP.1999 | [1999 IEEE International Conference on Acoustics, Speech, and Signal Processing. Proceedings. ICASSP99 (Cat. No.99CH36258)](https://ieeexplore.ieee.org//xpl/conhome/6110/proceeding) | Phoenix, AZ, USA |
| 1998 | 23 | 10.1109/ICASSP.1998 | [Proceedings of the 1998 IEEE International Conference on Acoustics, Speech and Signal Processing, ICASSP '98 (Cat. No.98CH36181)](https://ieeexplore.ieee.org//xpl/conhome/5518/proceeding) | Seattle, WA, USA |
| 1997 | 22 | 10.1109/ICASSP.1997 | [1997 IEEE International Conference on Acoustics, Speech, and Signal Processing](https://ieeexplore.ieee.org//xpl/conhome/4635/proceeding) | Munich, Germany |
| 1996 | 21 | 10.1109/ICASSP.1996 | [1996 IEEE International Conference on Acoustics, Speech, and Signal Processing Conference Proceedings](https://ieeexplore.ieee.org//xpl/conhome/3856/proceeding) | Atlanta, GA, USA |
| 1995 | 20 | 10.1109/ICASSP.1995 | [1995 International Conference on Acoustics, Speech, and Signal Processing](https://ieeexplore.ieee.org//xpl/conhome/3469/proceeding) | Detroit, MI, USA |
| 1994 | 19 | 10.1109/ICASSP.1994 | [Proceedings of ICASSP '94. IEEE International Conference on Acoustics, Speech and Signal Processing](https://ieeexplore.ieee.org//xpl/conhome/3104/proceeding) | Adelaide, SA, Australia |
| 1993 | 18 | 10.1109/ICASSP.1993 | [1993 IEEE International Conference on Acoustics, Speech, and Signal Processing](https://ieeexplore.ieee.org//xpl/conhome/1051/proceeding) | Minneapolis, MN, USA |
| 1992 | 17 | 10.1109/ICASSP.1992 | [[Proceedings] ICASSP-92: 1992 IEEE International Conference on Acoustics, Speech, and Signal Processing](https://ieeexplore.ieee.org//xpl/conhome/626/proceeding) | San Francisco, CA, USA |
| 1991 | 16 | 10.1109/ICASSP.1991 | [[Proceedings] ICASSP 91: 1991 International Conference on Acoustics, Speech, and Signal Processing](https://ieeexplore.ieee.org//xpl/conhome/540/proceeding) | Toronto, ON, Canada |
| 1991 | 15 | 10.1109/ICASSP.1991 | [International Conference on Acoustics, Speech, and Signal Processing](https://ieeexplore.ieee.org//xpl/conhome/132/proceeding) | Albuquerque, NM, USA |
| 1991 | 14 | 10.1109/ICASSP.1991 | [International Conference on Acoustics, Speech, and Signal Processing,](https://ieeexplore.ieee.org//xpl/conhome/805/proceeding) | Glasgow, UK |
| 1988 | 13 | 10.1109/ICASSP.1988 | [ICASSP-88., International Conference on Acoustics, Speech, and Signal Processing](https://ieeexplore.ieee.org//xpl/conhome/723/proceeding) | New York, NY, USA |
| 1987 | 12 | 10.1109/ICASSP.1987 | [ICASSP '87. IEEE International Conference on Acoustics, Speech, and Signal Processing](https://ieeexplore.ieee.org//xpl/conhome/8363/proceeding) | Dallas, TX, USA |
| 1986 | 11 | 10.1109/ICASSP.1986 | [ICASSP '86. IEEE International Conference on Acoustics, Speech, and Signal Processing](https://ieeexplore.ieee.org//xpl/conhome/8362/proceeding) | Tokyo, Japan |
| 1985 | 10 | 10.1109/ICASSP.1985 | [ICASSP '85. IEEE International Conference on Acoustics, Speech, and Signal Processing](https://ieeexplore.ieee.org//xpl/conhome/8361/proceeding) | Tampa, FL, USA |
| 1984 | 9 | 10.1109/ICASSP.1984 | [ICASSP '84. IEEE International Conference on Acoustics, Speech, and Signal Processing](https://ieeexplore.ieee.org//xpl/conhome/8372/proceeding) | San Diego, CA, USA |
| 1983 | 8 | 10.1109/ICASSP.1983 | [ICASSP '83. IEEE International Conference on Acoustics, Speech, and Signal Processing](https://ieeexplore.ieee.org//xpl/conhome/8371/proceeding) | Boston, MA, USA |
| 1982 | 7 | 10.1109/ICASSP.1982 | [ICASSP '82. IEEE International Conference on Acoustics, Speech, and Signal Processing](https://ieeexplore.ieee.org//xpl/conhome/8370/proceeding) | Paris, France |
| 1981 | 6 | 10.1109/ICASSP.1981 | [ICASSP '81. IEEE International Conference on Acoustics, Speech, and Signal Processing](https://ieeexplore.ieee.org//xpl/conhome/8369/proceeding) | Atlanta, GA,USA |
| 1980 | 5 | 10.1109/ICASSP.1980 | [ICASSP '80. IEEE International Conference on Acoustics, Speech, and Signal Processing](https://ieeexplore.ieee.org//xpl/conhome/8368/proceeding) | Denver, CO, USA |
| 1979 | 4 | 10.1109/ICASSP.1979 | [ICASSP '79. IEEE International Conference on Acoustics, Speech, and Signal Processing](https://ieeexplore.ieee.org//xpl/conhome/8367/proceeding) | Washington, DC, USA |
| 1978 | 3 | 10.1109/ICASSP.1978 | [ICASSP '78. IEEE International Conference on Acoustics, Speech, and Signal Processing](https://ieeexplore.ieee.org//xpl/conhome/8366/proceeding) | Tulsa, OK, USA |
| 1977 | 2 | 10.1109/ICASSP.1977 | [ICASSP '77. IEEE International Conference on Acoustics, Speech, and Signal Processing](https://ieeexplore.ieee.org//xpl/conhome/8365/proceeding) | Hartford, CT, USA |
| 1976 | 1 | 10.1109/ICASSP.1976 | [ICASSP '76. IEEE International Conference on Acoustics, Speech, and Signal Processing](https://ieeexplore.ieee.org//xpl/conhome/8364/proceeding) | Philadelphia, PA, USA |